## Домашнее задание № 5

#### Задание 1.
Обучите нейронную сеть решать шифр цезаря.
Что надо сделать:
1.Написать алгоритм шифра цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
2.Сделать нейронную сеть
3.Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
4.Проверить качество

#### Задание 2.
Выполнить практическую работу из лекционного ноутбука.
а) построить RNN-ячейку на основе полносвязных слоев
б) применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”


In [35]:
import time
import torch
from torch.autograd import Variable

In [36]:
### для начала пишем алгоримт шифра цезаря

def caesar(data, key, mode):
    alphabet = 'abcdefghijklmnopqrstuvwyzABCDEFGHIJKLMNOPQRSTUVWYZ'
    new_data = ''
    for c in data:
        # Shift character
        index = alphabet.find(c)
        if index == -1:
            # Character not found
            new_data += c
        else:
            # Shift it based on key and mode
            new_index = index + key if mode == 1 else index - key
            new_index %= len(alphabet)
            new_data += alphabet[new_index:new_index+1]
    # Return the ciphered text
    return new_data

In [37]:
### генерируем случайные предложения
import random

nouns = ("puppy", "car", "rabbit", "girl", "monkey")
verbs = ("runs", "hits", "jumps", "drives", "barfs") 
adv = ("crazily", "dutifully", "foolishly", "merrily", "occasionally")
adj = ("adorable", "clueless", "dirty", "odd", "stupid")

In [38]:
phrase_original = []
phrase_ciphered = []

for i in range(0,100):
    num = random.randrange(0,5)
    key = random.randrange(0,5)
    
    sentence = nouns[num] + ' ' + verbs[num] + ' ' + adv[num] + ' ' + adj[num]
    
    phrase_original.append(sentence)
    ciphered = caesar(sentence, key, 1)
    phrase_ciphered.append(ciphered)

#### Строим модель:

In [39]:
text_original = [[c for c in ph] for ph in phrase_original if type(ph) is str]
text_ciphered = [[c for c in ph] for ph in phrase_ciphered if type(ph) is str]

In [40]:
CHARS = set('abcdefghijklmnopqrstuvwyzABCDEFGHIJKLMNOPQRSTUVWYZ ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [41]:
MAX_LEN = 50

X = torch.zeros((len(text_original), MAX_LEN), dtype=int)
for i in range(len(text_original)):
    for j, w in enumerate(text_original[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
        
Y = torch.zeros((len(text_ciphered), MAX_LEN), dtype=int)
for i in range(len(text_ciphered)):
    for j, w in enumerate(text_ciphered[i]):
        if j >= MAX_LEN:
            break
        Y[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [43]:
class RNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # RNN
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        return out

In [44]:
# Create RNN
input_dim = 28    # input dimension
hidden_dim = 100  # hidden layer dimension
layer_dim = 1     # number of hidden layers
output_dim = 10   # output dimension

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

In [45]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [54]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

RuntimeError: input must have 3 dimensions, got 2

Источники:
1. https://www.kaggle.com/kanncaa1/recurrent-neural-network-with-pytorch
2. https://medium.com/x8-the-ai-community/building-a-recurrent-neural-network-from-scratch-f7e94251cc80 
3. https://towardsdatascience.com/deep-learning-4-embedding-layers-f9a02d55ac12 
4. https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526 
5. https://stackoverflow.com/questions/29938804/generate-random-sentences-in-python
6. https://medium.com/vacatronics/caesar-cipher-in-python-98d06e98989d
7. https://medium.com/towards-artificial-intelligence/building-neural-networks-from-scratch-with-python-code-and-math-in-detail-i-536fae5d7bbf
8. https://www.reddit.com/r/learnmachinelearning/comments/ct5ctq/new_to_ml_want_to_train_machine_to_decrypt_caesar/
9. https://greydanus.github.io/2017/01/07/enigma-rnn/